# Word Embededing based Text classification 

## by Yash Tripathi


In this document we will be following the Text classification method using Word embededing and DNN Models to classify the data and get the Accuracy Result.
It will be then compared with the other Predictions done by my other team memnbers and then we will choose the most aplicable model from the set.

Team Members:-
[@Yash Tripahti](https://drkakku.github.io/GoV2/todo.html)
[@Hitesh Goyal](https://drkakku.github.io/GoV2/todo.html)
[@Ashwin Iyer](https://drkakku.github.io/GoV2/todo.html)
[@Rishma ](https://drkakku.github.io/GoV2/todo.html)
[@Rahul Dash](https://drkakku.github.io/GoV2/todo.html)





The basic Pipeline that i will be following in this case will be :- 

- Split the data into text (X) and labels (Y)
- Preprocess X
- Create a word embedding matrix from X
- Create a tensor input from X
- Train a deep learning model using the tensor inputs and labels (Y)
- Make predictions on new data



In [5]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.initializers import Constant
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [6]:
%pip install pyspellchecker
from spellchecker import SpellChecker
#import numpy as np
import pandas as pd
import re
import string
from nltk.corpus import stopwords
#nltk.download("stopwords")

Note: you may need to restart the kernel to use updated packages.


In [7]:
stop_words = stopwords.words("english")

In [8]:
############################
### Get Data ##
train = pd.read_csv("../input/train.csv",index_col=0)
test = pd.read_csv("../input/test.csv",index_col=0)
validation = pd.read_csv("../input/validation.csv",index_col=0)

In [9]:
def factorise(df):
    df.label =  pd.factorize(df['label'])[0].astype(np.uint16)

In [10]:
factorise(train)
factorise(test)
factorise(validation)

In [11]:
validation.head()

,title,text,label
43465,white house petition calling ariz primary inve...,petition calling obama administration investig...,0
13278,civil war ii fourth turning intensifying part,posted november jimq part one article laid cas...,0
5764,homeland security chairman hillary mishandling...,homeland security chairman hillary mishandling...,0
21826,trump criticizes congress s move weakening eth...,washington reuters us presidentelect donald tr...,1
60667,yahoo taps x desktop search,search engine giant yahoo tapped pasadenabased...,1


We can use this cleaning function to clean the dataset 

It takes a lot of time and may be unnessary as it will most probabily not contribute much to the accuracy
```py
spell = SpellChecker()
def correct_spellings(x, spell=spell):
    """correct the missplled words of a given tweet"""
    x = x.split()
    misspelled = spell.unknown(x)
    result = map(lambda word : spell.correction(word) if word in  misspelled else word, x)
    return " ".join(result)

def tweets_cleaning(x, correct_spelling=True, remove_emojis=True, remove_stop_words=True):
    """Apply function to a clean a tweet"""
    x = x.lower().strip()
    # romove urls
    url = re.compile(r'https?://\S+|www\.\S+')
    x = url.sub(r'',x)
    # remove html tags
    html = re.compile(r'<.*?>')
    x = html.sub(r'',x)
    # remove punctuation
    operator = str.maketrans('','',string.punctuation) #????
    x = x.translate(operator)
    if correct_spelling:
        x = correct_spellings(x)
    if remove_emojis:
        x = x.encode('ascii', 'ignore').decode('utf8').strip()
    if remove_stop_words:
        x = ' '.join([word for word in x.split(' ') if word not in stop_words])
    return x


## APPLY the cleaning function to the text column
df['cleaned_tweets'] = df['text'].apply(tweets_cleaning)
```

In [13]:
trainX,trainY = xySplit(train)
testX,testY = xySplit(test)
validationX,validationY = xySplit(validation)

gert a distribution of the tokens

In [28]:
total_words = pd.concat([trainX,validationX])
np.shape(total_words)
total_words.head()

,title,text
6457,ashton kutcher rescues sex trafficking victims...,christopher ashton kutcher wellknown figure am...
52107,florida crowd attacks police officer attemptin...,citizenry respect law law enforcement officers...
55294,oracle profit rises software demand,san francisco reuters oracle corp tuesday repo...
27913,erdogan pope say phone call attempts change je...,ankara reuters turkish president tayyip erdoga...
59719,allrounder mcgrath,glenn mcgrath thoroughbred fast bowler decade ...


In [30]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import text

# Limit on the number of features to K features.
TOP_K = 20000

# Limit on the length of text sequences. 
# Sequences longer than this will be truncated.
# and less than it will be padded
MAX_SEQUENCE_LENGTH = 150

class CustomTokenizer:
    def __init__(self, train_texts):
        self.train_texts = train_texts
        self.tokenizer = Tokenizer(num_words=TOP_K)
        
    def train_tokenize(self):
        # Get max sequence length.
        max_length = len(max(self.train_texts , key=len))
        self.max_length = min(max_length, MAX_SEQUENCE_LENGTH)
    
        # Create vocabulary with training texts.
        self.tokenizer.fit_on_texts(self.train_texts)
        
    def vectorize_input(self, tweets):
        # Vectorize training and validation texts.
        
        tweets = self.tokenizer.texts_to_sequences(tweets)
        # Fix sequence length to max value. Sequences shorter than the length are
        # padded in the beginning and sequences longer are truncated
        # at the beginning.
        tweets = sequence.pad_sequences(tweets, maxlen=self.max_length, truncating='post',padding='post')
        return tweets

tokenizer = CustomTokenizer(train_texts = total_words['text'])
# fit o the train
tokenizer.train_tokenize()
tokenized_train = tokenizer.vectorize_input(trainX['text'])
tokenized_val = tokenizer.vectorize_input(validationX['text'])
tokenized_test = tokenizer.vectorize_input(testX['text'])

import os
import tqdm
import requests
import zipfile
URL = "http://nlp.stanford.edu/data/glove.42B.300d.zip"

def fetch_data(url=URL, target_file='../input/embedings/glove.zip', delete_zip=False):
    #if the dataset already exists exit
    if os.path.isfile(target_file):
        print("datasets already downloded :) ")
        return

    #download (large) zip file
    #for large https request on stream mode to avoid out of memory issues
    #see : http://masnun.com/2016/09/18/python-using-the-requests-module-to-download-large-files-efficiently.html
    print("**************************")
    print("  Downloading zip file")
    print("  >_<  Please wait >_< ")
    print("**************************")
    response = requests.get(url, stream=True)
    #read chunk by chunk
    handle = open(target_file, "wb")
    for chunk in tqdm.tqdm(response.iter_content(chunk_size=512)):
        if chunk:  
            handle.write(chunk)
    handle.close()  
    print("  Download completed ;) :") 
    #extract zip_file
    zf = zipfile.ZipFile(target_file)
    print("1. Extracting {} file".format(target_file))
    zf.extractall()
    if delete_zip:
        print("2. Deleting {} file".format(dataset_name+".zip"))
        os.remove(path=zip_file)

fetch_data()


In [31]:
glove_file = "../input/embedings/glove.6B.300d.txt"
import tqdm

EMBEDDING_VECTOR_LENGTH = 300 # = 300
def construct_embedding_matrix(glove_file, word_index):
    embedding_dict = {}
    with open(glove_file,'r') as f:
        for line in f:
            values=line.split()
            # get the word
            word=values[0]
            if word in word_index.keys():
                # get the vector
                vector = np.asarray(values[1:], 'float32')
                embedding_dict[word] = vector
    ###  oov words (out of vacabulary words) will be mapped to 0 vectors

    num_words=len(word_index)+1
    #initialize it to 0
    embedding_matrix=np.zeros((num_words, EMBEDDING_VECTOR_LENGTH))

    for word,i in tqdm.tqdm(word_index.items()):
        if i < num_words:
            vect=embedding_dict.get(word, [])
            if len(vect)>0:
                embedding_matrix[i] = vect[:EMBEDDING_VECTOR_LENGTH]
    return embedding_matrix
  
embedding_matrix =  construct_embedding_matrix(glove_file, tokenizer.tokenizer.word_index)


100%|██████████| 236079/236079 [00:00<00:00, 689569.34it/s]


In [17]:

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
    

In [32]:
model=Sequential()

embedding=Embedding(len(tokenizer.tokenizer.word_index)+1, # number of unique tokens
                    EMBEDDING_VECTOR_LENGTH, #number of features
                    embeddings_initializer=Constant(embedding_matrix), # initialize 
                    input_length=MAX_SEQUENCE_LENGTH, 
                    trainable=False)
                    

model.add(embedding)
model.add(Dropout(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.5))
model.add(Dense(1, activation='sigmoid'))

optimzer = Adam(clipvalue=0.5)
model.compile(optimizer=optimzer,loss='binary_crossentropy', metrics=['acc'])

In [33]:
import tensorflow as tf
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)



In [36]:

# fit the model
history = model.fit(tokenized_train, trainY, 

                    batch_size=32, 
                    epochs=20, 
                    validation_data=(tokenized_val,validationY), 
                    verbose=1,
                    callbacks=[tensorboard_callback])

Epoch 1/20
1242/1242 [==============================] - 315s 253ms/step - loss: 0.1929 - acc: 0.9292 - val_loss: 0.1912 - val_acc: 0.9253
Epoch 2/20
1242/1242 [==============================] - 312s 251ms/step - loss: 0.1799 - acc: 0.9337 - val_loss: 0.1412 - val_acc: 0.9533
Epoch 3/20
1242/1242 [==============================] - 318s 256ms/step - loss: 0.1495 - acc: 0.9479 - val_loss: 0.1335 - val_acc: 0.9557
Epoch 4/20
1242/1242 [==============================] - 317s 255ms/step - loss: 0.1410 - acc: 0.9508 - val_loss: 0.1242 - val_acc: 0.9595
Epoch 5/20
1242/1242 [==============================] - 311s 251ms/step - loss: 0.1301 - acc: 0.9546 - val_loss: 0.1245 - val_acc: 0.9573
Epoch 6/20
1242/1242 [==============================] - 316s 254ms/step - loss: 0.1213 - acc: 0.9576 - val_loss: 0.1205 - val_acc: 0.9593
Epoch 7/20
1242/1242 [==============================] - 308s 248ms/step - loss: 0.1170 - acc: 0.9593 - val_loss: 0.1305 - val_acc: 0.9590
Epoch 8/20
1242/1242 [============

KeyboardInterrupt: 

In [37]:
model.save("../models/yash/lstm64_half.h5")

model.save("../models/yash/lstm64_half")



INFO:tensorflow:Assets written to: ../models/yash/lstm64_half/assets


INFO:tensorflow:Assets written to: ../models/yash/lstm64_half/assets


ValueError: Message third_party.py.keras.protobuf.SavedMetadata exceeds maximum protobuf size of 2GB: 2408393954

In [ ]:
tokenized_train[0]

array([ 4280, 11336,  4758,  1241,    43,  3741,   617,  2338,  1502,
          10,  4500,  5770,  6644,  3354,  4975,   200,   309,  1120,
       15035,  3005,    80,  9350,  1397,   378,  1866,    75,  1241,
       10824,   729,  3971,  1028, 11172,  1453,   606,  4550,     7,
         647,   935,  1294,  1431,    41,  4220,  1852,  3416,   258,
        4220,   213,  3782,  6212,  2045], dtype=int32)

In [38]:
evaluate = model.evaluate(tokenized_test,testY,callbacks=[tensorboard_callback])

335/335 [==============================] - 23s 68ms/step - loss: 6.3764 - acc: 0.0426
